### Entrenamiento clase

In [3]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.multiclass import OneVsRestClassifier
#from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pandas as pd
import pickle
import itertools
from collections import Counter

In [4]:
class trainter:
    def __init__ (self):
        self.df = pd.DataFrame()
        self.train_data = pd.DataFrame()
        self.target_data = pd.DataFrame()
        self.randomforest = RandomForestClassifier(n_estimators=100, random_state=1)
        self.forest = DecisionTreeClassifier()
    
    #Carga el fichero con el nombre indicado y nombra las columnas
    def load_file(self,fileName):
        self.df = pd.read_csv(fileName, sep=',', header=None)
        self.df = self.df
        self.df.columns = ['timeStamp', 'Px', 'Py', 'heat', 'Exp1','Eyp1','Exp2','Eyp2', 'Ex1', 'Ey1', 'Eh1', 'Ex2', 'Ey2', 'Eh2', 'Ex3', 'Ey3', 'Eh3', 'Ex4', 'Ey4','Eh4','Ex5', 'Ey5','Eh5','Ex6', 'Ey6','Eh6',"VKey","HKey","Shooting"]
        self.df[['Eh1','Eh2','Eh3','Eh4','Eh5','Eh6']] = self.df[['Eh1','Eh2','Eh3','Eh4','Eh5','Eh6']].astype(float) 
    
    #procesa las columnas correspondientes al trainda
    def set_train_data(self,columns):
        print("setting train_data")
        t = self.df[columns].copy()
        print("\t enemies info" )
        enemiesInfo = t[['Ex1', 'Ey1', 'Ex2', 'Ey2', 'Ex3', 'Ey3','Ex4', 'Ey4','Ex5', 'Ey5','Ex6', 'Ey6','Eh1','Eh2','Eh3','Eh4','Eh5','Eh6']].copy()
        enemiesInfo = enemiesInfo.apply(self.histo2DRow, axis = 1)
        print("\t power up info" )
        powerUpInfo = t[['Exp1','Eyp1','Exp2','Eyp2']].copy()
        powerUpInfo = powerUpInfo.apply(self.histo2DRow,axis = 1)
        print("\t other info" )
        otherInfo = t.drop(['Ex1', 'Ey1', 'Ex2', 'Ey2', 'Ex3', 'Ey3','Ex4', 'Ey4','Ex5', 'Ey5','Ex6', 'Ey6','Eh1','Eh2','Eh3','Eh4','Eh5','Eh6','Exp1','Eyp1','Exp2','Eyp2'], axis=1)
        powerUpInfo = self.rename_trainData(powerUpInfo)
        enemiesInfo = self.rename_trainData(enemiesInfo)
        self.train_data = pd.concat([otherInfo,powerUpInfo,enemiesInfo],axis = 1)
        
        #Concatenar instancias
        print("concatenar instancias")
        df = pd.concat([self.train_data,self.target_data],axis = 1)
        df = self.concatenar_estados(df)
        self.train_data = df
        self.train_data = self.train_data.drop(['VKey', 'HKey', 'Shooting'],axis=1)
        
        
    #salva el dataframe con la info de train_data
    def save_train_data(self,fileName):
        print("saving train_data")
        np.save(fileName,self.train_data)
    
    #carga el dataframe con la info de train_data. Esto es para no procesar todo el dataframe cada vez.
    def load_train_data(self,fileName):
        print("loading train_data")
        self.train_data = np.load(fileName)
    
    def set_target_data(self,columns):
        print("Setting target_data")
        self.target_data = self.df[columns].copy()
        self.target_data["VKey"] = self.target_data["VKey"].map(self.transformaEje)
        self.target_data["HKey"] = self.target_data["HKey"].map(self.transformaEje)
        self.target_data["Shooting"] = self.target_data["Shooting"].map(self.transformaDisparo)
    
    def histo2DRow(self,row):
        x = row.filter(regex=('Ex'))
        y = row.filter(regex=('Ey'))
        w = row.filter(regex=('Eh'))
        
        xedges = np.linspace(-2,2,9)
        yedges = np.linspace(0,10,15)
        if len(w >0):
            H, xedges, yedges = np.histogram2d(x, y, bins=(xedges, yedges), weights=w)
        else:
            H, xedges, yedges = np.histogram2d(x, y, bins=(xedges, yedges))
        H = H.T  # Let each row list bins with common y range.
        return pd.Series(H.reshape(-1))

    def concatenar_estados(self,dataframe):
        dfAnterior = dataframe.shift(1)
        dfAnterior.rename(columns=lambda x: str(x)+"-1", inplace=True)
        dfAnterior2 = dataframe.shift(2)
        dfAnterior2.rename(columns=lambda x: str(x)+"-2", inplace=True)
        df = pd.concat([dfAnterior2,dfAnterior,dataframe],axis=1)
        df = df.fillna(999)
        return df
        
    def rename_trainData(self,DataFrame):
        oldNames = list(range(len(DataFrame.columns)))
        newNames = ["("+str(round(x,2))+ ","+ str(round(y,2)) +")" for x, y in itertools.product(np.linspace(0,10,15),np.linspace(-2,2,9))]
        print("Renombrado")
        DataFrame = DataFrame.rename(columns=dict(zip(oldNames,newNames)))
        return DataFrame
    
    def train(self):
        #self.randomforest = self.randomforest.fit(self.train_data, self.target_data.astype(str))
        self.forest = self.forest.fit(self.train_data, self.target_data.astype(str))
        
        
    def save_forest(self,fileName):
        pickle.dump(self.forest, open(fileName, 'wb'))
    
    def transformaEje(self,value):
        if value == "UpArrow" or value == "RightArrow":
            return 1
        elif value == "DownArrow" or value == "LeftArrow":
            return -1
        else:
            return 0
        
    def transformaDisparo(self,value):
        if value:
            return 1
        else:
            return 0
        
    def __str__(self):
        return str(self.df)

In [5]:
t = trainter()
t.load_file('gameStates.csv')
t.set_target_data(["VKey","HKey","Shooting"])
t.set_train_data(['Px', 'Py', 'heat', 'Exp1','Eyp1','Exp2','Eyp2', 'Ex1', 'Ey1', 'Eh1', 'Ex2', 'Ey2', 'Eh2', 'Ex3', 'Ey3', 'Eh3', 'Ex4', 'Ey4','Eh4','Ex5', 'Ey5','Eh5','Ex6', 'Ey6','Eh6'])
t.train()
t.save_forest("randomForest.sav")
print("ok")

C:\Users\Pablo Alejos\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2821: DtypeWarning: Columns (0,28) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Setting target_data
setting train_data
	 enemies info


C:\Users\Pablo Alejos\Anaconda3\lib\site-packages\numpy\lib\function_base.py:804: RuntimeWarning: invalid value encountered in greater_equal
  not_smaller_than_edge = (sample[:, i] >= edges[i][-1])


	 power up info
	 other info
Renombrado
Renombrado
concatenar instancias
ok


In [234]:
for n in t.train_data.columns:
    print(n)

Px-2
Py-2
heat-2
(0.0,-2.0)-2
(0.0,-1.5)-2
(0.0,-1.0)-2
(0.0,-0.5)-2
(0.0,0.0)-2
(0.0,0.5)-2
(0.0,1.0)-2
(0.0,1.5)-2
(0.0,2.0)-2
(0.71,-2.0)-2
(0.71,-1.5)-2
(0.71,-1.0)-2
(0.71,-0.5)-2
(0.71,0.0)-2
(0.71,0.5)-2
(0.71,1.0)-2
(0.71,1.5)-2
(0.71,2.0)-2
(1.43,-2.0)-2
(1.43,-1.5)-2
(1.43,-1.0)-2
(1.43,-0.5)-2
(1.43,0.0)-2
(1.43,0.5)-2
(1.43,1.0)-2
(1.43,1.5)-2
(1.43,2.0)-2
(2.14,-2.0)-2
(2.14,-1.5)-2
(2.14,-1.0)-2
(2.14,-0.5)-2
(2.14,0.0)-2
(2.14,0.5)-2
(2.14,1.0)-2
(2.14,1.5)-2
(2.14,2.0)-2
(2.86,-2.0)-2
(2.86,-1.5)-2
(2.86,-1.0)-2
(2.86,-0.5)-2
(2.86,0.0)-2
(2.86,0.5)-2
(2.86,1.0)-2
(2.86,1.5)-2
(2.86,2.0)-2
(3.57,-2.0)-2
(3.57,-1.5)-2
(3.57,-1.0)-2
(3.57,-0.5)-2
(3.57,0.0)-2
(3.57,0.5)-2
(3.57,1.0)-2
(3.57,1.5)-2
(3.57,2.0)-2
(4.29,-2.0)-2
(4.29,-1.5)-2
(4.29,-1.0)-2
(4.29,-0.5)-2
(4.29,0.0)-2
(4.29,0.5)-2
(4.29,1.0)-2
(4.29,1.5)-2
(4.29,2.0)-2
(5.0,-2.0)-2
(5.0,-1.5)-2
(5.0,-1.0)-2
(5.0,-0.5)-2
(5.0,0.0)-2
(5.0,0.5)-2
(5.0,1.0)-2
(5.0,1.5)-2
(5.0,2.0)-2
(5.71,-2.0)-2
(5.71,-1.5)-2
(5.71

In [235]:
t.target_data

,VKey,HKey,Shooting
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
5,0,-1,0
6,0,0,0
7,-1,0,0
8,-1,0,0
9,0,1,0
